# 📘 P1.2.2.5 – Python Error Handling
## Topic: Graceful Error Recovery

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Clean up resources properly when errors occur
- Use `finally` blocks to guarantee cleanup
- Implement fallback mechanisms
- Implement retry logic with exponential backoff
- Understand context managers (`with` statements)

## 🔄 What is Graceful Recovery?
Graceful recovery means your system **continues operating even when something fails**.

Instead of crashing, you:
- Return a safe default
- Retry failed operations
- Switch to a backup approach
- Warn the user but keep going

## 🧹 Critical: Resource Cleanup on Errors
**The core of graceful recovery: NEVER leave resources open when errors occur.**

When an error happens in the middle of an operation, you must:
1. **Catch the error** (don't crash)
2. **Clean up resources** (close files, connections, release locks)
3. **Continue or return gracefully** (not abruptly exit)

**Examples of resources that need cleanup:**
- Open files (must close even if error occurs)
- Database connections (must disconnect/rollback)
- Network sockets (must close)
- Locks (must release)
- Temporary data (must delete)

In [ ]:
# ❌ BAD: No cleanup on error - resource left open!
def bad_file_processing():
    file = open("data.txt", "r")
    try:
        data = file.read()
        result = int(data)  # Might fail
        return result
    except ValueError:
        print("Invalid data")
        # ERROR: file is still open! Resource leak!


# ✅ GOOD: Cleanup with finally
def good_file_processing():
    file = open("data.txt", "r")
    try:
        data = file.read()
        result = int(data)
        return result
    except ValueError:
        print("Invalid data")
        return None
    finally:
        file.close()  # ALWAYS executed, even on error
        print("File closed (cleanup done)")

print("Running bad_file_processing():")
bad_file_processing()

print("\nRunning good_file_processing():")
good_file_processing()

## ✅ Resource Cleanup Checklist

**When an error occurs in the middle of an operation, ensure you clean up:**

| Resource Type | Cleanup Action | Example |
|---|---|---|
| **Files** | Close file handles | `file.close()` |
| **Database Connections** | Rollback transaction + disconnect | `db.rollback()`, `db.close()` |
| **Temp Files** | Delete temporary files | `os.remove(temp_file)` |
| **Network Connections** | Close connection | `connection.close()` |

---

## 🔁 Retry Logic with Exponential Backoff
For transient failures (network glitches), retry automatically.
Wait longer between retries to avoid overwhelming the system.

In [ ]:
import time
import random

def call_api_with_retry(url, max_retries=3):
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt + 1}/{max_retries}: Calling {url}")
            if random.random() < 0.7:  # 70% failure rate for demo
                raise ConnectionError("Network timeout")
            return "Success: Data received"
        except ConnectionError as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"Failed. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Failed after {max_retries} attempts")
                return "Failed: Used cache or default data"

result = call_api_with_retry("https://api.example.com/data")
print(result)

### ✅ Key Takeaways
- Graceful recovery = continue running even when errors occur
- Resource cleanup in `finally` blocks prevents resource leaks
- Fallback mechanisms keep systems running
- Retry logic automatically handles temporary failures
- **In AI/ML:** Cleanup ensures data pipelines don't leave open files, model training doesn't crash mid-process, and inference endpoints gracefully return cached results on failure